In [6]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import tqdm
import psycopg2
import pandas as pd

In [7]:
conn = psycopg2.connect("host=localhost dbname=gradient_local user=postgres") 
cursor = conn.cursor()

get_items_query = "SELECT * FROM items;"
get_users_query = "SELECT * FROM users;"
get_fct_query = "SET constraint_exclusion = on; SELECT * FROM fct_hourly_metric;"

In [8]:
df = pd.read_sql(get_items_query, conn)
df.head()

/var/folders/0y/d_717f6d3wv7j3stmqzgt9_c0000gp/T/ipykernel_65740/3059927008.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(get_items_query, conn)


,item_key,created_at,user_id,bucket_key,type
0,d6a9a727-3c39-41bc-a46a-1027a85a0fac,2023-07-14 14:35:37,24549,tiny-texts-jk9apq,txt
1,24153e9f-c757-448a-b854-ade8b23feff5,2023-07-14 14:35:25,32109,tiny-images-jk9apq,img
2,5921d0f8-8989-495d-bf28-247743063659,2023-07-14 14:33:48,33961,tiny-texts-jk9apq,txt
3,fdc57820-5a49-4467-b8ce-44e814948d0e,2023-07-14 14:31:47,2295,tiny-texts-jk9apq,txt
4,e14c4b33-b1e5-42a4-920d-b1c10a5a5545,2023-07-14 14:31:02,26137,tiny-images-jk9apq,img
